In [1]:
from dotenv import load_dotenv
import openai
import os
from openai import AzureOpenAI
from IPython.display import display, HTML, JSON, Markdown, Image

load_dotenv()
AZURE_OPENAI_ENDPOINT=os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_KEY=os.getenv("AZURE_OPENAI_KEY")
AZURE_OPENAI_API_VERSION=os.getenv("AZURE_OPENAI_API_VERSION")
AZURE_OPENAI_GPT4o_DEPLOYMENT=os.getenv("AZURE_OPENAI_GPT4o_DEPLOYMENT")

client = openai.AzureOpenAI(
        azure_endpoint=AZURE_OPENAI_ENDPOINT,
        api_key=AZURE_OPENAI_KEY,
        api_version=AZURE_OPENAI_API_VERSION
)
print(f"Model: {AZURE_OPENAI_GPT4o_DEPLOYMENT}; API Version:{AZURE_OPENAI_API_VERSION}")
print("Azure OpenAI Chatbot is ready to use!")


Model: gpt-4o; API Version:2024-10-21
Azure OpenAI Chatbot is ready to use!


In [2]:
from langchain_openai import AzureChatOpenAI
llm = AzureChatOpenAI(
            deployment_name=AZURE_OPENAI_GPT4o_DEPLOYMENT,
            model=AZURE_OPENAI_GPT4o_DEPLOYMENT,
            openai_api_version=AZURE_OPENAI_API_VERSION,
            api_key= AZURE_OPENAI_KEY,
            azure_endpoint=AZURE_OPENAI_ENDPOINT,
            temperature=0,
            max_tokens=2000
            )




In [3]:
from langchain import PromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain.document_loaders import PyPDFLoader

fileName = "./data/Best-Practices-in-Change-Management-Full-Report-Digital-11th-Edition.pdf"
loader = PyPDFLoader(fileName)
pages = loader.load_and_split()
print(f"filename: {fileName}")
print("Number of pages: ", len(pages))

filename: ./data/Best-Practices-in-Change-Management-Full-Report-Digital-11th-Edition.pdf
Number of pages:  367


![Upload](./images/map_reduce.png)

In [4]:
map_prompt_template = """
                      Write a summary of this chunk of text that includes the main points and any important details.
                      {text}
                      """

map_prompt = PromptTemplate(template=map_prompt_template, input_variables=["text"])

combine_prompt_template = """
                      Write a concise summary of the following text delimited by triple backquotes.
                      Return your response in bullet points which covers the key points of the text.
                      ```{text}```
                      BULLET POINT SUMMARY:
                      """

combine_prompt = PromptTemplate(
    template=combine_prompt_template, input_variables=["text"]
)

In [5]:
map_reduce_chain = load_summarize_chain(
    llm,
    chain_type="map_reduce",
    map_prompt=map_prompt,
    combine_prompt=combine_prompt,
    return_intermediate_steps=True,
)

In [6]:
# This might take some time to run. For demo purposes, limit it to 10 pages.
map_reduce_outputs = map_reduce_chain.invoke({"input_documents": pages[100:110]})
print("Number of outputs: ", len(map_reduce_outputs))

Number of outputs:  3


In [7]:
display(Markdown(map_reduce_outputs['output_text']))

- The text discusses the value and measurement of change management in projects, based on studies from 2013, 2015, 2016, and 2019.
- Key benefits of change management include increased realization of project benefits, improved employee adoption, and enhanced engagement.
- Measurement methods include post-implementation reviews, surveys, feedback, assessments, and sustainability measures.
- Metrics for evaluating change management effectiveness include employee responses, project results, end-user adoption, and project success.
- The 2015 study identified five primary methods for measuring effectiveness: surveys, adoption observation, feedback, assessments, and eliminated challenges.
- In 2019, 51% of participants reported measuring change management effectiveness, using additional channels like employee surveys and industry data.
- Best practices focus on measuring change at the individual level, using metrics like compliance, usage, attendance, and engagement.
- Data sources for measurement include end users, stakeholders, project teams, and senior leaders, with timing varying from pre to post-project.
- Key strategies for measuring individual progress include observation, communication, tests, performance evaluations, and surveys.
- The 2015 study highlighted three key metrics: speed of adoption, ultimate utilization, and proficiency, with varying levels of success reported.
- Effective change management is linked to better outcomes in speed of adoption, ultimate utilization, and proficiency.
- Methods and metrics for measuring feedback, work output, proficiency, and ultimate utilization include interviews, performance evaluations, and compliance audits.
- Project success is evaluated through KPIs like schedule adherence, budget, intended outcomes, and employee usage.
- Common obstacles in change management include leadership and sponsorship issues, project management challenges, and employee resistance.
- The text emphasizes the importance of early and structured change management methodology to enhance project success.

In [8]:
import pandas as pd
from pathlib import Path as p
final_mp_data = []
for doc, out in zip(map_reduce_outputs["input_documents"], map_reduce_outputs["intermediate_steps"]):
    output = {}
    output["file_name"] = p(doc.metadata["source"]).stem
    output["file_type"] = p(doc.metadata["source"]).suffix
    output["page_number"] = doc.metadata["page"]
    output["chunks"] = doc.page_content
    output["concise_summary"] = out
    final_mp_data.append(output)

pdf_mp_summary = pd.DataFrame.from_dict(final_mp_data)
pdf_mp_summary = pdf_mp_summary.sort_values(
    by=["file_name", "page_number"]
)  # sorting the dataframe by filename and page_number
pdf_mp_summary.reset_index(inplace=True, drop=True)
pdf_mp_summary.head()

file_name file_type  page_number  \
0  Best-Practices-in-Change-Management-Full-Repor...      .pdf           95   
1  Best-Practices-in-Change-Management-Full-Repor...      .pdf           96   
2  Best-Practices-in-Change-Management-Full-Repor...      .pdf           97   
3  Best-Practices-in-Change-Management-Full-Repor...      .pdf           98   
4  Best-Practices-in-Change-Management-Full-Repor...      .pdf           99   

                                              chunks  \
0  Prosci \n®    \nBest Practices in Change Manag...   
1  96  •  © 2020 Prosci Inc. \nFigure 5.9 – Measu...   
2  Prosci \n®    \nBest Practices in Change Manag...   
3  98  •  © 2020 Prosci Inc. \n2. Observation and...   
4  Prosci \n®    \nBest Practices in Change Manag...   

                                     concise_summary  
0  The text discusses the value and measurement o...  
1  The text discusses methods for measuring chang...  
2  The text discusses best practices in change ma...  
3  The text outlines various methods used to meas...  
4  The text summarizes findings from a 2015 study...

In [9]:
index = 6
print("[Context]")
print(pdf_mp_summary["chunks"].iloc[index])

[Context]
Prosci 
®    
Best Practices in Change Management
© 2020 Prosci Inc.  •  101
3. Feedback and observation  
Participants collected feedback through interviews, 
focus groups and one-on-one conversations. 
Feedback was collected from key individuals 
including employees, managers, clients and change 
network members. Participants also gauged ultimate 
use by observing and monitoring employees in the 
new state.
4. Work output and performance 
Identified less than half as often as the previous 
finding, employee performance and output also 
determined ultimate use. Performance evaluations, 
project goals and updated key performance 
indicators (KPIs) were used. The quality and quantity 
of work output was also evaluated to determine how 
many employees were using the solution.
5. Timeline and process completion 
Participants tracked timeline, process and task 
completion often using checklists. Some said the time 
it took for employees to become efficient using a new 
solution c

In [10]:
print("\n\n [Simple Summary]")
print(pdf_mp_summary["concise_summary"].iloc[index])
print("\n\n [Page number]")
print(pdf_mp_summary["page_number"].iloc[index])
print("\n\n [Source: file_name]")
print(pdf_mp_summary["file_name"].iloc[index])



 [Simple Summary]
The text outlines methods and metrics used in change management to measure feedback, work output, proficiency, and ultimate utilization. Feedback was gathered through interviews, focus groups, and observations from employees, managers, clients, and change network members. Work output and performance were assessed using performance evaluations, project goals, and key performance indicators (KPIs). Timeline and process completion were tracked using checklists to measure efficiency in adopting new solutions. Proficiency was measured by analyzing work output, conducting surveys and assessments, and observing employee engagement. Ultimate utilization was measured through compliance audits, system usage tracking, and surveys to assess productivity, readiness, and user acceptance. Compliance was also gauged by monitoring workarounds and the extent of reverting to old practices.


 [Page number]
101


 [Source: file_name]
Best-Practices-in-Change-Management-Full-Report-Digi